In [1]:
#Import dependencies
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from Postgres_Login import username
from Postgres_Login import password

In [2]:
#Store CSV into DataFrame
csv_file = "Resources/netflix_dataset_EE.csv"
netflix_data_df = pd.read_csv(csv_file)
netflix_data_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [3]:
#Drop 'Movies', only want 'TV Shows'
new_netflix_data_df = netflix_data_df.loc[netflix_data_df['type'] == 'TV Show']
new_netflix_data_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
5,80163890,TV Show,Apaches,NaN,"Alberto Ammann, Eloy Azorín, Verónica Echegui,...",Spain,"September 8, 2017",2016,TV-MA,1 Season,"Crime TV Shows, International TV Shows, Spanis...",A young journalist is forced into a life of cr...
8,80117902,TV Show,Fire Chasers,NaN,NaN,United States,"September 8, 2017",2017,TV-MA,1 Season,"Docuseries, Science & Nature TV","As California's 2016 fire season rages, brave ..."
26,80244601,TV Show,Castle of Stars,NaN,"Chaiyapol Pupart, Jintanutda Lummakanon, Worra...",NaN,"September 7, 2018",2015,TV-14,1 Season,"International TV Shows, Romantic TV Shows, TV ...",As four couples with different lifestyles go t...


In [4]:
#Create new data with select columns (Title, Genres, No. of Seasons, TV Rating) 
#Add new column and name it 'Streaming Service'
clean_netflix_data_df = new_netflix_data_df[['title', 'listed_in', 'duration', 'rating']]
clean_netflix_data = clean_netflix_data_df.rename(columns = {"title" : "Title of Show",
                                                          "listed_in" : "Genres",
                                                          "duration" : "No. of Seasons",
                                                          "rating" : "TV Rating"})
final_netflix_data = clean_netflix_data.dropna()
final_netflix_data['Netflix'] = 'Yes'
final_netflix_data

,Title of Show,Genres,No. of Seasons,TV Rating,Netflix
2,Transformers Prime,Kids' TV,1 Season,TV-Y7-FV,Yes
3,Transformers: Robots in Disguise,Kids' TV,1 Season,TV-Y7,Yes
5,Apaches,"Crime TV Shows, International TV Shows, Spanis...",1 Season,TV-MA,Yes
8,Fire Chasers,"Docuseries, Science & Nature TV",1 Season,TV-MA,Yes
26,Castle of Stars,"International TV Shows, Romantic TV Shows, TV ...",1 Season,TV-14,Yes
...,...,...,...,...,...
6228,Kikoriki,Kids' TV,2 Seasons,TV-Y,Yes
6229,Red vs. Blue,"TV Action & Adventure, TV Comedies, TV Sci-Fi ...",13 Seasons,NR,Yes
6230,Maron,TV Comedies,4 Seasons,TV-MA,Yes
6232,A Young Doctor's Notebook and Other Stories,"British TV Shows, TV Comedies, TV Dramas",2 Seasons,TV-MA,Yes


In [7]:
final_netflix_data["No. of Seasons"] = final_netflix_data["No. of Seasons"].str.strip("Seasons").astype(str)
final_netflix_data

,Title of Show,Genres,No. of Seasons,TV Rating,Netflix
2,Transformers Prime,Kids' TV,1,TV-Y7-FV,Yes
3,Transformers: Robots in Disguise,Kids' TV,1,TV-Y7,Yes
5,Apaches,"Crime TV Shows, International TV Shows, Spanis...",1,TV-MA,Yes
8,Fire Chasers,"Docuseries, Science & Nature TV",1,TV-MA,Yes
26,Castle of Stars,"International TV Shows, Romantic TV Shows, TV ...",1,TV-14,Yes
...,...,...,...,...,...
6228,Kikoriki,Kids' TV,2,TV-Y,Yes
6229,Red vs. Blue,"TV Action & Adventure, TV Comedies, TV Sci-Fi ...",13,NR,Yes
6230,Maron,TV Comedies,4,TV-MA,Yes
6232,A Young Doctor's Notebook and Other Stories,"British TV Shows, TV Comedies, TV Dramas",2,TV-MA,Yes


In [5]:
#Connect to local database
rds_connection_string = f"{username}:{password}@localhost:5432/streaming_services"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [6]:
#Use pandas to load csv converted DataFrame into database
final_netflix_data.to_sql(name='netflix2', con=engine, if_exists='append', index=False)
pd.read_sql_query('select * from netflix2', con=engine).head()

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "Netflix" of relation "netflix2" does not exist
LINE 1: ...f Show", "Genres", "No. of Seasons", "TV Rating", "Netflix")...
                                                             ^

[SQL: INSERT INTO netflix2 ("Title of Show", "Genres", "No. of Seasons", "TV Rating", "Netflix") VALUES (%(Title of Show)s, %(Genres)s, %(No. of Seasons)s, %(TV Rating)s, %(Netflix)s)]
[parameters: ({'Title of Show': 'Transformers Prime', 'Genres': "Kids' TV", 'No. of Seasons': '1 Season', 'TV Rating': 'TV-Y7-FV', 'Netflix': 'Yes'}, {'Title of Show': 'Transformers: Robots in Disguise', 'Genres': "Kids' TV", 'No. of Seasons': '1 Season', 'TV Rating': 'TV-Y7', 'Netflix': 'Yes'}, {'Title of Show': 'Apaches', 'Genres': 'Crime TV Shows, International TV Shows, Spanish-Language TV Shows', 'No. of Seasons': '1 Season', 'TV Rating': 'TV-MA', 'Netflix': 'Yes'}, {'Title of Show': 'Fire Chasers', 'Genres': 'Docuseries, Science & Nature TV', 'No. of Season